In [ ]:
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set

# data the maze
# ---------------
# indir = indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pastime1/11.2024_StapleTIS/simulations/flat/sim_istar0br2108/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar2cosbr_2108/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarz_g5/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarz_slopeg5_2702/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar0t/"

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_2108/"
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_0209/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_br/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_add_0502/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_add_br2305/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar0br10intf/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar0br6dintf/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar0br15dintf/"
# indir = "/mnt/tw06_biommeda_pastime1/11.2024_StapleTIS/simulations/maze2D/sim_istarmaze_kick2708/"
# indir = "/mnt/tw06_biommeda_pastime1/11.2024_StapleTIS/simulations/maze2D/sim_/"
zero_minus_one = False
inputfile = indir + "logging.log"


import os
import glob
os.chdir(indir)
print(os.getcwd())

folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell250k_0506/005']


In [ ]:
# !!! last lines !!!  allow to speed up this notebook
# pe.set_orders(load=False...)  -> 1st time you run the code, this will store npy files
# pe.set_orders(load=True...)  -> next time you run the code, you can read npy files

# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)
    # read order parameters order.txt/order.npy into path ensemble object
    #pe.set_orders(load=False, acc_only=True, save=False) # if saving doesn't work
    #### CHANGE HERE ####
    # pe.set_orders(load=False, acc_only=True, save=True) # for the 1st time
    # pe.set_orders(load=True, acc_only=True) # for the next times, save=True/False is not important

In [ ]:
from tistools import block_error_analysis_staple, get_local_probs, get_global_probs_from_dict
import logging
import sys
from datetime import datetime
from contextlib import redirect_stdout, redirect_stderr

# Create a log file with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"block_error_analysis_{timestamp}.log"

# Configure logging to file
file_handler = logging.FileHandler(log_filename, mode='w')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# Create logger
log = logging.getLogger()
log.setLevel(logging.INFO)
log.handlers = [file_handler]  # Replace any existing handlers

# Write initial message to log file only
log.info(f"Starting block error analysis at {timestamp}")

# Open the log file for capturing all output
with open(log_filename, 'a') as log_file, redirect_stdout(log_file), redirect_stderr(log_file):
    # Run the block error analysis for StapleTIS
    block_error_analysis_staple(pathensembles, interfaces, 1, pl=False)
    
    # Calculate REPPTIS probabilities for comparison
    repptisploc = []
    for i, pe in enumerate(pathensembles):
        # REPPTIS p_loc
        repptisploc.append(get_local_probs(pe, tr=False))

    _, _, reppfull = get_global_probs_from_dict(repptisploc)
    
    # Write REPPTIS results to log file
    print("\nREPPTIS global crossing probabilities:")
    for i, p in enumerate(reppfull):
        print(f"Interface {i}: {p:.8f}")

# Log file path is the only output to notebook
log.info(f"Analysis complete - results saved to {log_filename}")


In [1]:
from pathlib import Path
from tistools import load_path_ensembles, block_error_analysis

# BASE_DIR = Path("/mnt/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations")
BASE_DIR = Path("/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/")
simulations = [
    # ("flat_w-walls", "brownian-gamma5/30k-cycles"),
    # ("flat_w-walls", "langevin-gamma5"),   
    # ("flat_w-walls", "newtonian"),                          
    # ("cosbump2-walls", "brownian"),
    # ("cosbump2-walls", "langevin-gamma5"),                 
    # ("cosbump3-walls", "brownian"),  
    # ("cosbump3-walls", "langevin-gamma5"), 
    # ("cosdip2-walls", "brownian"), 
    # ("cosdip2-walls", "langevin-gamma5"), 
    # ("cosbumpmeta-walls", "brownian"), 
    # ("cosbumpmeta-walls", "brownian/fine_intf"),
    # ("cosbumpmeta-walls", "brownian/intf_shift"),
    # ("cosbumpmeta-walls", "langevin_gamma5"),
    # ("cosbumpmeta-walls", "langevin_gamma5/fine_intf"),
    # ("cosbumpmeta-walls", "langevin_gamma5/intf_shift"),
    # ("cosdipmeta-walls", "brownian"), 
    # ("cosdipmeta-walls", "langevin_gamma5"),
    ("", "")       
]

indir_list = [BASE_DIR / sim_type / config / "REPPTIS" for sim_type, config in simulations]

for indir in indir_list:
    path_ensembles, interfaces = load_path_ensembles(indir,load=True)
    block_error_analysis(path_ensembles, interfaces, interval=10, load=False)




Weights of the different paths:
wRMR = 0
wRML = 14332
wLMR = 14408
wLML = 0
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 1.0
pLML = 0.0
Local crossing probabilities:
p2R = 0.501322199025748
p2L = 0.4986778009742519
Weights of the different paths:
wRMR = 0
wRML = 14239
wLMR = 14501
wLML = 0
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 1.0
pLML = 0.0
Local crossing probabilities:
p2R = 0.5045581071677105
p2L = 0.4954418928322895
Weights of the different paths:
wRMR = 0
wRML = 14312
wLMR = 14428
wLML = 0
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 1.0
pLML = 0.0
Local crossing probabilities:
p2R = 0.502018093249826
p2L = 0.49798190675017395
Collect tau
ensemble 0 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/000
ensemble 1 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/001
ensemble 2 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS/002
ensemble